## Web-scraping ##

In [ ]:
from bs4 import BeautifulSoup as bs
import requests

tutors = []
for index in range(1, 50):

    page = requests.get('https://repetit.ru/repetitors/matematika/?page=' + str(index))

    if not page.ok:
        print('Incorrect data')
        break
    else:
        html = bs(page.text, 'html.parser')


    for person in html.find_all('div', class_="teacher-card"):
        for link in person.find_all('a', class_="teacher-name"):
            tutors.append("https://repetit.ru" + link['href'])

info = []
for link in tutors:
    person = bs(requests.get(link).text, 'html.parser')
    person_info = []

    name = person.find('h1', class_='teacher-name').text.replace('  ', ' ')
    person_info.append(name)
    # print(name)
    age = person.find('div', class_='row age').find('div', class_='col-8').find('span', class_='hide-in-edit-mode').text
    person_info.append(age)
    # print(age)
    experience = person.find('div', class_='row experience').find('div', class_='col-8').find('span', class_='hide-in-edit-mode').text
    person_info.append(experience)
    # print(experience)

    education = person.find('div', class_='row education').find('div', class_='col-8 educations')
    try:
        edu = education.find('a').text
    except:
        edu = education.find('span').text
    person_info.append(edu)
    # print(edu)

    star_rating = len(person.find('div', class_='star-rating').find_all('i', class_='icon-star w10'))
    person_info.append(star_rating)
    # print(star_rating)

    try:
        verification = person.find('div', class_='features').find('i')['data-show-on-click']
    except:
        verification = 'false'
    person_info.append(verification)
    # print(verification)

    price = int(person.find('p', class_='price').find('strong').text.replace(' ', ''))
    person_info.append(price)
    # print(price)

    subj_list = []
    subjects = person.find_all('div', class_='col subject-name')
    for subject in subjects:
        subj_list.append(subject.find('h4').text)
    person_info.append(', '.join(subj_list))
    # print(subj_list)

    rating_list = person.find('div', class_='statistic in-nav').find_all('div', class_='col-3')
    total_rating = rating_list[0].find('h5').text
    person_info.append(total_rating)

    students = int(rating_list[3].find('h5').text.split()[0])
    person_info.append(students)
    # print(total_rating, students)

    info.append(person_info)

import pandas as pd
df = pd.DataFrame(info)
df.columns = ['Name', 'Age', 'Experience', 'Education', 'Star rating', 'Verification', 'Price', 'Subjects', 'Total rating', 'Students']
df.to_excel('tutors.xlsx', index=False)
